# Function1

In [ ]:
with tf.variable_scope(scope):
            init_w = tf.random_normal_initializer(0., 0.3)
            init_b = tf.constant_initializer(0.1)
            
            x_h1 = tf.layers.dense(inputs = self.state_input, units = neuron_num, \
                   activation = tf.nn.tanh, kernel_initializer=init_w, \
                   bias_initializer=init_b, trainable=Trainable) 
            x_h2 = tf.layers.dense(inputs = self.state_input, units = neuron_num, \
                   activation = tf.nn.tanh, kernel_initializer=init_w, \
                   bias_initializer=init_b, trainable=Trainable)
            
            output = tf.layers.dense(inputs = x_h2, units = self.n_actions, \
                   activation = tf.nn.softmax, kernel_initializer=init_w, \
                   bias_initializer=init_b, trainable=Trainable)#輸出為個行為的機率，因此使用softmax
            
        
        return output #輸出‘不同動作’對應的Q值

# Function2

In [ ]:
# discount episode rewards
        Vt = np.zeros_like(self.past_reward, dtype=np.float64)
        Vt_temp = 0
        for t in reversed(range(0, len(self.past_reward))):
            Vt_temp = self.past_reward[t] + Vt_temp * self.gamma
            Vt[t] = Vt_temp

        # normalize episode rewards
        
        Vt -= np.mean(Vt)
        Vt /= np.std(Vt)
        
        return Vt

# Function3

In [ ]:
action_one_hot = self.action_one_hot.copy()
action_one_hot[action] = 1
self.past_state.append(current_state)
self.past_action.append(action_one_hot)
self.past_reward.append(reward)

# Function4

In [ ]:
act_proba = self.sess.run(self.act_proba, feed_dict={self.state_input: current_state[np.newaxis, :]})
        """
        根據神經網路輸出個動作的機率當作權重做隨機抽樣選出動作。一樣舉超級瑪莉的例子，如果輸出
        [上, 下, 左, 右]的機率分別為[0.3, 0.1, 0.2, 0.4]，那做隨機抽樣就分別有30%, 10%, 
        20%和40%的機率抽到上、下、左、右，因為這樣的做法已具有隨機性因此不需要用epsilon來控制
        行為的隨機程度來增加探索。
        """
        self.action = np.random.choice(range(act_proba.shape[1]), p=act_proba.ravel())  
   
        return self.action

# Function5

In [ ]:
#用reward算出更新向量
Vt = self.calculate_Vt()
#print(np.array(self.past_action).shape)
#print(np.array(self.past_action))
#將整個episode的資料放進來做更新
self.sess.run(self.train, feed_dict={
     self.state_input: np.vstack(self.past_state),  # shape=[None, n_state]
     self.real_action: np.array(self.past_action),  # shape=[None, n_actions]
     self.Vt: np.array(Vt)  # shape=[None, ]
})
#更新完後將記憶庫清空
self.past_state, self.past_action, self.past_reward = [], [], []    